# Imports and Settings for Website

In [2]:
import os
import selenium
import time
from PIL import Image
import io
import requests

# download image
import urllib.request

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
# Action chains is for complex mouse button actions
from selenium.webdriver.common.action_chains import ActionChains

import user
qanvast_user = user.Qanvast['user']
qanvast_pass = user.Qanvast['pass']

## Set up Selenium for Chrome

In [4]:
# Check About Chrome website version to download right selenium driver
from selenium.webdriver import Chrome

# # Set up incognito
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("--incognito")

chrome_prefs = {}
chrome_options.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

driver = webdriver.Chrome('C:\scrap\chromedriver', options=chrome_options)

# driver = webdriver.Chrome('C:\scrap\chromedriver')

## Put in starting point of scrape

In [5]:
# Specify Search URL/Start Point
search_url = "https://qanvast.com/sg/interior-design-singapore"
driver.get(search_url)

# Go to login
login_page = WebDriverWait(driver, 10).until(presence_of_element_located((By.CLASS_NAME, "_300aJmN3k9YoDqpg76oEXf")))
login_page.click()

# Fill credentials
login_user = driver.find_element_by_xpath("(//input[@class='_2vxAdHKMPYf8ByTMp5N_Jk'])[1]")
login_pass = driver.find_element_by_xpath("(//input[@class='_2vxAdHKMPYf8ByTMp5N_Jk'])[2]")
login_user.send_keys(qanvast_user)
login_pass.send_keys(qanvast_pass)
# Login
driver.find_element_by_class_name("_3tOEYVFn9u8qPkn9P4lMVL").click()
WebDriverWait(driver, 3)
# Go to first element
first_box = driver.find_element_by_xpath("(//li[@class='slide selected'])")
ActionChains(driver).move_to_element(first_box).perform()
first_box.click()


### CHECKS  - Confirm touching the correct elements

In [17]:
ul_box = driver.find_element_by_xpath("(/html/body/div[8]/div/div/div/div/div/div[2]/div[1]/div[2]/div/div/ul)")

In [18]:
curr_li = ul_box.find_elements_by_tag_name('li')
n_steps = len(curr_li)

# Extracting links of the images
for i in range(n_steps):
    li = curr_li[i]
    per_li_div = li.find_elements_by_tag_name('div')[0]
    try:
        per_li_div_div = per_li_div.find_elements_by_tag_name('div')[0]
    except:
        continue
    per_li = per_li_div_div.find_elements_by_tag_name('img')[0]
    print(per_li.get_attribute('alt'))
    print(per_li.get_attribute('src'))

The Adara, The Roomakers, Modern, Contemporary, Bathroom, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505545/f33edb6892/720-width
The Adara, The Roomakers, Modern, Contemporary, Living Room, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505548/2d45ff1ae6/720-width
The Adara, The Roomakers, Modern, Contemporary, Living Room, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505549/273f684b98/720-width
The Adara, The Roomakers, Modern, Contemporary, Dining Room, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505550/c597440882/720-width
The Adara, The Roomakers, Modern, Contemporary, Bedroom, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505546/75f559b856/720-width
The Adara, The Roomakers, Modern, Contemporary, Bathroom, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505545/f33edb6892/720-width
The Adara, The Roomakers, Modern, Contemporary, Living Room, Condo
https://d1hy6t2xeg0mdl.cloudfront.net/image/505548/2d45ff1ae6/720-width


In [16]:
d = driver.find_element_by_xpath("(//div[@class='carousel carousel-slider'])")
e = d.find_element_by_xpath("(//div[@class='slider-wrapper axis-horizontal'])")
ul_box = e.find_element_by_xpath("(//ul[@class='slider animated'])")
curr_li = ul_box.find_elements_by_tag_name('li')
n_steps = len(curr_li)

proj = []
# Extracting links of the images
for i in range(n_steps):
    li = curr_li[i]
    per_li_div = li.find_elements_by_tag_name('div')[0]
    try:
        per_li_div_div = per_li_div.find_elements_by_tag_name('div')[0]
    except:
        continue
    per_li = per_li_div_div.find_elements_by_tag_name('img')[0]
    print(per_li.get_attribute('alt'))
    print(per_li.get_attribute('src'))
#     proj.append(temp)

Yishun Street 21, Metier Planner, Contemporary, Living Room, HDB
https://d1hy6t2xeg0mdl.cloudfront.net/image/505643/3fe6277d90/standard
Yishun Street 21, Metier Planner, Contemporary, Living Room, HDB
https://d1hy6t2xeg0mdl.cloudfront.net/image/505643/3fe6277d90/standard


In [152]:
print(ul_box.get_attribute('style'))

transform: translate3d(-100%, 0px, 0px); transition-duration: 350ms;


# Start Scrape

### Set Variables

In [14]:
# Data Structures
# unique = set()
check_unique = False
count = 1000
room_types = ["Balcony", "Bathroom", "Bedroom", "Commercial", "Dining Room", "Garden", "Kitchen", "Living Room", "Study"]

In [9]:
import ast
unique = set()
with open("id.txt", "r") as f:
    unique = ast.literal_eval(f.read())

In [40]:
def extraction():
    """ Function to extract all img links into a list"""
    # Get the ul list
    for _ in range(2):
#         ### OUTDATED ULBOX
#         d = driver.find_element_by_xpath("(//div[@class='carousel carousel-slider'])")
#         e = d.find_element_by_xpath("(//div[@class='slider-wrapper axis-horizontal'])")
#         ul_box = e.find_element_by_xpath("(//ul[@class='slider animated'])")
        # IMAGES BLOCKED ulbox
        ul_box = driver.find_element_by_xpath("(/html/body/div[7]/div/div/div/div/div/div[2]/div[1]/div[2]/div/div/ul)")

#         # Exact ulbox xpath - IMAGES NOT BLOCKED
#         ul_box = driver.find_element_by_xpath("(/html/body/div[8]/div/div/div/div/div/div[2]/div[1]/div[2]/div/div/ul)")
        curr_li = ul_box.find_elements_by_tag_name('li')
        n_steps = len(curr_li)
        #print(f"Current project images: {n_steps} ")

        # Loop through slideshow to load images
        svgA = driver.find_element_by_xpath("//div[@class='XiDtBVvTZHPCGUIbhtue _2-Y_1udykLwVs81bFrVldC']")
        #print("Looping through slideshow.")
        wait = WebDriverWait(driver, 0.5)
        for _ in range(n_steps):
            driver.execute_script("arguments[0].click();", svgA)
            
    current_project_name = driver.find_element_by_class_name("_1eGrec2mPwl8VD4p7AQjGI").text
    print(f"{current_project_name}. {n_steps} Images.", end =" ")

    proj = []
    # Extracting links of the images
    for i in range(n_steps):
        li = curr_li[i]
        per_li_div = li.find_elements_by_tag_name('div')[0]
        try:
            per_li_div_div = per_li_div.find_elements_by_tag_name('div')[0]
        except:
            continue
        per_li = per_li_div_div.find_elements_by_tag_name('img')[0]
        temp = {per_li.get_attribute('alt'):per_li.get_attribute('src')}
        proj.append(temp)
#         print(proj)
    return proj

In [ ]:
while True:
    # Extract list of names and tags
    check_unique = False
    print()
    print(f"Project {count}: ")
    curr = extraction()
    # next project
    next_proj = driver.find_element_by_xpath("//div[@class='_3SU9oNAvAeM4pvx3i45gx1']")
    driver.execute_script("arguments[0].click();", next_proj)
    print(f" .", end =" ")
    while check_unique == False:
        for i in curr:
            for k,v in i.items():
                prop = k.split(', ')[-1]
                identity = v.split('/')[-3]
                print(f" {identity},", end =" ")
                # Stop Extraction if exists
                if identity in unique:
                    check_unique = True
                else:
                    unique.add(int(identity))
                    
                # Else Start Extracting
                if prop == 'Commercial':
                    filename = 'scraped/' + prop + '/' + identity+' '+k +'.jpg'
                else:
                    if len(k.split(', '))>1:
                        room = k.split(', ')[-2]
                        # There may be photos without room name
                        if room in room_types:
                            filename = 'scraped/' + room + '/' + identity+' '+k +'.jpg'
                        else:
                            filename = 'scraped/Commercial/' + identity+' '+k +'.jpg'
                    else:
                        filename = 'scraped/Commercial/' + identity+' '+k +'.jpg'
#                 print(filename)
#                 print(v)
                urllib.request.urlretrieve(v, filename)
        break
#     if check_unique == False:
    count+=1
    print(f"..Scraped.", end =" ")
#     else:
#         print(f"..Scraped before, skipping...", end =" ")


Project 2122: 
. 12 Images.  .  421626,  421638,  421630,  421624,  421628,  421621,  421623,  421639,  421636,  421627,  421626,  421638, ..Scraped. 
Project 2123: 
. 12 Images.  .  421606,  421618,  421612,  421603,  421611,  421608,  421613,  421610,  421604,  421616,  421606,  421618, ..Scraped. 
Project 2124: 
. 12 Images.  .  421592,  421597,  421598,  421602,  421587,  421596,  421595,  421582,  421589,  421591,  421592,  421597, ..Scraped. 
Project 2125: 
Tampines Street 32. 12 Images.  .  421517,  421516,  421519,  421507,  421524,  421506,  421523,  421498,  421518,  421511,  421517,  421516, ..Scraped. 
Project 2126: 
. 12 Images.  .  421458,  421456,  421485,  421490,  421488,  421483,  421468,  421480,  421476,  421463,  421458,  421456, ..Scraped. 
Project 2127: 
. 12 Images.  .  421431,  421439,  421438,  421424,  421436,  421430,  421437,  421432,  421434,  421448,  421431,  421439, ..Scraped. 
Project 2128: 
Circuit Road. 9 Images.  .  421251,  421252,  421255,  42125

### Checking if elements extracted is the right information

In [26]:
name = list(curr[0].keys())[0]

'Gem Residences, Swiss Interior Design, Contemporary, Bathroom, Condo'

In [35]:
for i in curr:
    for k,v in i.items():
        prop = k.split(', ')[-1]
        identity = v.split('/')[-3]
        if prop == 'Commercial':
            filename = 'scraped/' + prop + '/' + identity+' '+k +'.jpg'
        else:
            room = k.split(', ')[-2]
            filename = 'scraped/' + room + '/' + identity+' '+k +'.jpg'
        urllib.request.urlretrieve(v, filename)

In [13]:
# Have to check if they images are already scraped
name = "scraped/Dining Room/filename.jpg"
urllib.request.urlretrieve("https://d1hy6t2xeg0mdl.cloudfront.net/image/505310/795b8810c7/1080-width", name)

('scraped/Dining Room/filename.jpg',
 <http.client.HTTPMessage at 0x2781ed21438>)

In [196]:
ul_box = driver.find_element_by_xpath("(//ul[@data-reactid='.2.$modal.0.0.1.1.0.1.0.1.0'])")

In [197]:
# CHECK: Printing of attributes.
from pprint import pprint
attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', ul_box)
pprint(attrs)

{'class': 'slider animated',
 'data-reactid': '.2.$modal.0.0.1.1.0.1.0.1.0',
 'style': 'transform: translate3d(-100%, 0px, 0px); transition-duration: '
          '350ms;'}


In [198]:
curr_li = ul_box.find_elements_by_tag_name('li')
print(f"Current project images: {len(curr_li)} ")

Current project images: 12 


In [199]:
# Times out and causes per_li_div itself to change elements
for i in range(len(curr_li)):
    
    li = curr_li[i]
    per_li_div = li.find_elements_by_tag_name('div')[0]
    print(per_li_div.get_attribute('data-reactid'))
    try:
        per_li_div_div = per_li_div.find_elements_by_tag_name('div')[0]
    except:
        continue
    per_li = per_li_div_div.find_elements_by_tag_name('img')[0]
    print(per_li.get_attribute('src'))

.2.$modal.0.0.1.1.0.1.0.1.0.$itemKey9clone/=1$9.$9
https://d1hy6t2xeg0mdl.cloudfront.net/image/505107/b437315f1b/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey0/=1$0.$0
https://d1hy6t2xeg0mdl.cloudfront.net/image/505117/d2d626ad67/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey1/=1$1.$1
https://d1hy6t2xeg0mdl.cloudfront.net/image/505112/6a6613de1b/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey2/=1$2.$2
https://d1hy6t2xeg0mdl.cloudfront.net/image/505106/f6d79b4869/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey3/=1$3.$3
https://d1hy6t2xeg0mdl.cloudfront.net/image/505115/c6418169f5/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey4/=1$4.$4
https://d1hy6t2xeg0mdl.cloudfront.net/image/505116/7e141d7db7/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey5/=1$5.$5
https://d1hy6t2xeg0mdl.cloudfront.net/image/505114/f5798b2218/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1:$itemKey6/=1$6.$6
https://d1hy6t2xeg0mdl.cloudfront.net/image/505103/0e291382ed/1080-width
.2.$modal.0.0.1.1.0.1.0.1.0.1

In [52]:
x = driver.find_element_by_tag_name("a")

first_elem = driver.find_element_by_xpath("(//ul[@data-reactid='.0.2.0.1.5.0.0.$listitems.1:$15735.0.1.0.1.0'])[1]")

In [22]:
# find list of links that is related to photo
x = driver.find_elements_by_css_selector("[href*='/sg/photos']");

# usual click is via webdriverclick, if ElementClickInterceptedException : x[0].click()
# use javascript click

In [ ]:
data-reactid='.3.$modal.0.0.1.1.0.1.0.1.0'"]

In [ ]:
current_ul = driver.find_element_by_xpath("//ul[@class='slider animated' and @data-reactid='.3.$modal.0.0.1.1.0.1.0.1.0']")

#### Misc. Selenium Commands

In [26]:
driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 't') 

In [25]:
ActionChains(driver).key_down(Keys.ALT).send_keys('d').perform()
ActionChains(driver).key_down(Keys.ENTER).perform()
ActionChains(driver).key_up(Keys.ALT).key_up(Keys.ENTER).perform()

In [29]:
import pickle
pickle.dump( driver.get_cookies() , open("cookies.pkl","wb")) # saving cookies

In [30]:
#  Deletes all cookies
driver.delete_all_cookies()

In [106]:
# Removing previous cache
#open tab
driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 't') 
driver.get('chrome://settings/clearBrowserData')
driver.find_element_by_xpath('//settings-ui').send_keys(Keys.ENTER)
clearBrowsingDataConfirm
driver.find_element_by_xpath('//settings-ui').send_keys(Keys.ENTER)
driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'w') 

In [ ]:
driver.switch_to.active_element

# RANDOM WORKINGS BELOW
---------------------------------------------------------------------------------------------------------------------------

In [58]:
current_ul = driver.find_element_by_xpath("//ul[@class='slider animated']")

In [34]:
storage = {}
current_ul.tag_name

'ul'

In [152]:
for num, li in enumerate(curr_li):
    per_li_div = li.find_elements_by_tag_name('div')[num]
    print(f"{type(per_li_div)}")
    if type(per_li_div) == 'selenium.webdriver.remote.webelement.WebElement':
        per_li_div_div = per_li_div.find_elements_by_tag_name('div')
    else:
        #print(f"length is {len(per_li_div)}")
        per_li_div_div = per_li_div.find_elements_by_tag_name('div')[0]
    per_li = per_li_div_div.find_elements_by_tag_name('img')[0]    
    print(per_li.get_attribute('src'))

<class 'selenium.webdriver.remote.webelement.WebElement'>
https://d1hy6t2xeg0mdl.cloudfront.net/image/505260/bc5824a2c1/720-width
<class 'selenium.webdriver.remote.webelement.WebElement'>


IndexError: list index out of range

In [141]:
for num, li in enumerate(curr_li):
    per_li_div = li.find_elements_by_tag_name('div')[num]
    per_li_div_div = per_li_div.find_elements_by_tag_name('div')
    if len(per_li_div_div) !=1:
        per_li_div_div = per_li_div_div[0]
    try:
        per_li = per_li_div_div.find_elements_by_tag_name('img')[0]
    #except JavascriptError:
    except:
        x = per_li_div_div.find_elements_by_css_selector("*")
        for element in x:
            outerhtml = element.get_attribute('outerHTML ') # to extract outerHTML 
            tag_value=outerhtml.split('',1)[0] # to extract first word
            print("element found with tag value = " + tag_value)
#         attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', per_li_div_div)
#         pprint(attrs)
        
    print(per_li.get_attribute('alt'))

AttributeError: 'list' object has no attribute 'find_elements_by_css_selector'

In [115]:
x= ul_box.find_elements_by_tag_name('li')[0].find_elements_by_tag_name('div')[0].find_elements_by_tag_name('div')[0].find_elements_by_tag_name('img')[0]

attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', x)
pprint(attrs)

{'alt': 'Tampines North Drive 1, Fifth Avenue Interior, Modern, Scandinavian, '
        'Bathroom, HDB',
 'data-reactid': '.2.$modal.0.0.1.1.0.1.0.1.0.$itemKey9clone/=1$9.$9.0.0',
 'src': 'https://d1hy6t2xeg0mdl.cloudfront.net/image/505260/bc5824a2c1/720-width'}


In [46]:
j = curr_li[0].find_element_by_xpath("//div/div/img")

In [47]:
j.get_attribute('alt')

'Save to Qanvast Board'

In [108]:
for li in curr_li:
    print(li.find_element_by_xpath("//div/div/img").get_attribute('alt'))

Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board
Save to Qanvast Board


In [76]:
d = driver.find_element_by_tag_name("img")

In [79]:
d.get_attribute('src')

'https://googleads.g.doubleclick.net/pagead/viewthroughconversion/959875665/?value=0&guid=ON&script=0'

In [77]:
for k in d:
    print(k.get_attribute('src'))

TypeError: 'WebElement' object is not iterable

In [62]:
k = driver.find_element_by_xpath("(//div[@class='_1LqapI10fxVkMKAyzTxjvZ'])/div/div/ul")


In [63]:
curr_li = k.find_elements_by_tag_name('li')

In [61]:
driver.find_element_by_class_name('_1LqapI10fxVkMKAyzTxjvZ')

<selenium.webdriver.remote.webelement.WebElement (session="464b6985c4275793577cdba47093a175", element="bddae0c6-0471-451d-9ee5-f9ad1d335066")>

In [28]:
driver.find_element_by_tag_name('img').get_attribute('src')

'https://googleads.g.doubleclick.net/pagead/viewthroughconversion/959875665/?value=0&guid=ON&script=0'

In [14]:

#link = driver.find_element_by_link_text('Load More Arguments')
ActionChains(driver).move_to_element(x[1]).perform()
x[1].click()

In [19]:
#link = driver.find_element_by_link_text('Load More Arguments')
ActionChains(driver).move_to_element(x[1]).perform()
x[1].click()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="/sg/photos/country-esquire-dining-room-504731" target="_blank" data-reactid=".0.2.0.1.5.0.0.$listitems.1:$15739.0.1.0.1.0.$itemKey0clone/=1$0.$0.0.0">...</a> is not clickable at point (224, 585). Other element would receive the click: <li class="slide selected" data-reactid=".0.2.0.1.5.0.0.$listitems.1:$15739.0.1.0.1.0.$itemKey0clone/=1$0">...</li>
  (Session info: chrome=90.0.4430.72)


In [6]:
x[0].get_attribute("href")

'https://qanvast.com/sg/photos/country-esquire-dining-room-504731'

In [78]:

driver.execute_script("arguments[0].click();", x[0])


In [112]:
#selector = "div[class^=carousel carousel-slider]"
x = driver.find_elements_by_css_selector("//div[@class^='carousel']")

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=89.0.4389.114)


In [21]:
for i in range(50):
    print(x[i].get_attribute("href"))

TypeError: 'WebElement' object does not support indexing

In [23]:
for i in range(50):
    print(x[i].get_attribute("href"))

https://qanvast.com/sg/photos/country-esquire-dining-room-504731
https://qanvast.com/sg/photos/country-esquire-dining-room-504731
https://qanvast.com/sg/photos/country-esquire-dining-room-504731
https://qanvast.com/sg/photos/ghim-moh-link-living-room-504712
https://qanvast.com/sg/photos/ghim-moh-link-living-room-504712
https://qanvast.com/sg/photos/clementi-avenue-3-living-room-504697
https://qanvast.com/sg/photos/clementi-avenue-3-living-room-504697
https://qanvast.com/sg/photos/canberra-street-living-room-504673
https://qanvast.com/sg/photos/canberra-street-living-room-504673
https://qanvast.com/sg/photos/pasir-ris-drive-1-living-room-504427
https://qanvast.com/sg/photos/pasir-ris-drive-1-living-room-504427
https://qanvast.com/sg/photos/ang-mo-kio-avenue-4-living-room-504413
https://qanvast.com/sg/photos/ang-mo-kio-avenue-4-living-room-504413
https://qanvast.com/sg/photos/parc-riviera-living-room-504403
https://qanvast.com/sg/photos/parc-riviera-living-room-504403
https://qanvast.com

IndexError: list index out of range

In [106]:
Path('.')

WindowsPath('.')

In [107]:
# Make Dirs
from pathlib import Path
Path('./scraped').mkdir(parents=True, exist_ok=True) # any missing parents are created 


In [ ]:
# Dashboard shows by list, Click the first element
driver.FindElements(By.ClassName("drop"))[1].Click();

In [ ]:
def scroll_to_end(wd, scroll_point):  
        wd.execute_script(f"window.scrollTo(0, {scroll_point});")
        time.sleep(3)    

In [ ]:
image_urls = set()
image_count = 0
number_results = 0

for i in range(1,20):
    scroll_to_end(driver, i*1000)
    time.sleep(5)
    thumb = wd.find_elements_by_css_selector("img._2zEKz")
    time.sleep(5)
    for img in thumb:
        print(img)
        print(img.get_attribute('src'))
        image_urls.add(img.get_attribute('src'))
        image_count = len(image_urls)
        number_results = image_count
        time.sleep(.5)
    print(f"Found: {number_results} search results. Extracting links...")

In [49]:
x = [driver.find_element_by_tag_name("ul")]


.send_keys(qanvast_pass)

driver.find_element_by_id (“ID”).send_keys(“password”)


SyntaxError: invalid syntax (<ipython-input-49-743b5766320e>, line 4)

In [65]:
k = driver.find_element_by_xpath("(//li[@class='slide selected'])")
ActionChains(driver).move_to_element(k).perform()
k.click()

In [66]:
ActionChains(driver).move_to_element(k).perform()
k.click()

In [ ]:
# Dashboard shows by list, Click the first element
driver.FindElements(By.ClassName("drop"))[1].Click();
#link = driver.find_element_by_link_text('Load More Arguments')

In [ ]:
# Login process 
login = wait.until(presence_of_element_located((By.CLASS_NAME, "_300aJmN3k9YoDqpg76oEXf")))
login.click()
first_result.get_attribute


In [4]:
#This example requires Selenium WebDriver 3.13 or newer
with webdriver.Chrome() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get("https://google.com/ncr")
    driver.find_element(By.NAME, "q").send_keys("cheese" + Keys.RETURN)
    first_result = wait.until(presence_of_element_located((By.CSS_SELECTOR, "h3>div")))
    print(first_result.get_attribute("textContent"))
  

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
